In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# read the data matrix
X = pd.read_csv('masked_matrix.csv').to_numpy()
orig_X = pd.read_csv('original_matrix.csv').to_numpy()

In [ ]:
# PARATUCK2 imputer
def PARATUCK2(X, n_LatCom, regParam):
    
    # parameters and random initialization
    maxIter = 200; eps = 1e-5; n, m = X.shape; p, q = n_LatCom
    A = np.random.randn(n, p) / p
    R = np.random.randn(p, q) 
    B = np.random.randn(m, q) / q
    
    # binary matrix of observed and not observed values
    Z = ~np.isnan(X); Z = Z.astype(np.float64, copy=False)
    
    # ALS algorithm
    X = np.nan_to_num(X, copy=True)
    M = np.zeros((p*q, p*q))
    N = np.zeros((p, q))
    RMSE = [np.linalg.norm(Z * (X - np.matmul(np.matmul(A, R), B.T)), 'fro') / np.sqrt(np.sum(Z))]
    for iter in range(maxIter):
        # update A
        for i, zi in enumerate(Z):
            A[i] = np.linalg.solve(np.matmul(R, np.matmul(np.matmul(B.T, np.matmul(np.diag(zi), B)), R.T)) + 
                                   regParam * np.eye(p), np.matmul(R, np.matmul(B.T, X[i].T))).T
        # update R   
        for k, zk in enumerate(Z):
            M = M + np.kron(np.matmul(B.T, np.matmul(np.diag(zk), B)), np.outer(A[k], A[k]))
            N = N + np.outer(A[k].T, np.matmul(X[k], B))
        R_vec = np.linalg.solve(M, N.flatten(order='F'))
        R = R_vec.reshape((p, q), order='F')
        # update B
        for j, zj in enumerate(Z.T):
            B[j] = np.linalg.solve(np.matmul(R.T, np.matmul(np.matmul(A.T, np.matmul(np.diag(zj), A)), R)) + 
                                   regParam * np.eye(q), np.matmul(R.T, np.matmul(A.T, X[:, j]))).T
        # stop criterion
        RMSE.append(np.linalg.norm(Z * (X - np.matmul(np.matmul(A, R), B.T)), 'fro') / np.sqrt(np.sum(Z)))
        if abs(RMSE[iter+1]-RMSE[iter]) <= eps:
            break
    # imputed matrix
    X_hat = np.matmul(np.matmul(A, R), B.T)
    
    return X_hat, A, R, B

In [ ]:
# call the function
X_hat, A, R, B = PARATUCK2(X, (5, 7), 200)
print(X_hat)

In [ ]:
# claculate validation error
x_hat = X_hat[np.logical_and(np.isnan(X), ~np.isnan(orig_X))]
x = orig_X[np.logical_and(np.isnan(X), ~np.isnan(orig_X))]

np.median(np.divide(np.abs((x - x_hat)), x))